In [1]:
load("skeinslib.sage")

In [2]:
M = matrix(ZZ, 2, [7, 2, 3, 1])

In [39]:
def compute_reduced_matrix_mod(gamma, shell_level, interactive_flag, base_level = 1, base_data=None):
    '''
    Computes the reduced matrix of relations at a given shell level, as well as
    its co-rank, which is an estimate of the dimension of the empty part of the
    skein module

    Returns a tuple (A, A_reduced, dim_estimates), where A is the matrix and
    dim_estimates is the list of estimated dimensions for shell levels <= to the
    shell level parameter.

    Works recursively: the relation matrix for level n is built up as

                [[  X  | 0 ]
                 [---------]
                 [relations]]

    where X is the matrix of relations for level n-1, in REF, obtained from a
    recursive call to the function, and the list relations is a list of
    relations at level n that were not already included at level n-1.

    Supports recursion to a specified base_level, with base_data the results of
    a previous call to the function (that is, a tuple of length 3). Usually this
    would be loaded as a sage object saved from a previous session. If the
    recursion base is unspecified then the default is recurse to shell_level 1
    and compute this data from scratch.
    '''

    # Base case.
    if shell_level == base_level:
        if base_level == 1: # Recursing all the way to level 1
            # For each shell level, compute #{lattice points}.
            N = (2*shell_level + 1)*(shell_level + 1) - shell_level
            # And the number of points in previous level.
            M = (2*shell_level - 1)*(shell_level) - shell_level + 1
            if interactive_flag:
                print("Calculating relations for level %d (%d lattice points) ..." % (shell_level, N))

            # Get the relations new to this shell level.
            relations = get_new_relations_empty(gamma, shell_level, order_by_shell_level)
            if interactive_flag:
                print("Found %d (non-independent) relations.\n" % len(relations))
            # Form a relation matrix, and reduce it.
            A = matrix(K, relations, sparse=True, immutable=True)
            A_reduced = A.rref()

            # Get the spanning set
            ordering = order_by_shell_level(shell_level)
            spanning_set = get_spanning_set(A_reduced, ordering, shell_level)

            # Base of the recursion: return the reduced matrix and spanning set.
            dim_estimate = len(spanning_set)
            if interactive_flag:
                print("Dimension estimate for empty skein part at level %d: %d.\n\nVisualisation:\n" % (shell_level, dim_estimate))
                print_generators(shell_level, spanning_set, order_by_shell_level)

            return (A, A_reduced, [dim_estimate])

        else: # Recursing to a higher level and picking up where we left off.
            if interactive_flag:
                print("Using previosuly calculated relations for level %d..." % (shell_level))
            return base_data

    else:
        # For each shell level, compute #{lattice points}.
        N = (2*shell_level + 1)*(shell_level + 1) - shell_level
        # And the number of points in previous level.
        M = (2*shell_level - 1)*(shell_level) - shell_level + 1
        if interactive_flag:
            print("Calculating relations for level %d (%d lattice points) ..." % (shell_level, N))

        # Get the relations new to this shell level.
        relations = get_new_relations_empty(gamma, shell_level, order_by_shell_level)
        if interactive_flag:
            print("Found %d (non-independent) relations. Reducing ..." % len(relations))
        # Get the relation matrix for the previous shell level, recursively.
        prev_data = compute_reduced_matrix(gamma, shell_level - 1, interactive_flag, base_level, base_data)
        A_old = prev_data[1]
        dimensions = prev_data[2]

        # Handle the case when there were no relations in the previous level
        if len(A_old.rows()) == 0:
            A = matrix(K, relations, sparse=True, immutable=True)
        else:
            # Use the new relations and old, reduced matrix to build the relations
            # matrix for this shell level, and reduce.
            Zeros_right = zero_matrix(K, A_old.nrows(), N - M, sparse=True)
            A_upper = block_matrix([[A_old, Zeros_right]])
            A_lower = matrix(K, relations, sparse=True, immutable=True)
            A = block_matrix([[A_upper], [A_lower]])
            
        if not shell_level == 5:
            A_reduced = A.rref()
        else:
            A_reduced = A

        # Get the spanning set
        ordering = order_by_shell_level(shell_level)
        spanning_set = get_spanning_set(A_reduced, ordering, shell_level)

        # Add the dimension estimate to the list, print output, and return
        dim_estimate = len(spanning_set)
        dimensions.append(dim_estimate)
        if interactive_flag:
            print("Dimension estimate for empty skein part at level %d: %d.\n\nVisualisation:\n" % (shell_level, dim_estimate))
            print_generators(shell_level, spanning_set, order_by_shell_level)

        return (A, A_reduced, dimensions)

In [5]:
data = compute_reduced_matrix_mod(M, 6, True)

Calculating relations for level 6 (85 lattice points) ...
Found 39 (non-independent) relations. Reducing ...
Calculating relations for level 5 (61 lattice points) ...
Found 21 (non-independent) relations. Reducing ...
Calculating relations for level 4 (41 lattice points) ...
Found 8 (non-independent) relations. Reducing ...
Calculating relations for level 3 (25 lattice points) ...
Found 1 (non-independent) relations. Reducing ...
Calculating relations for level 2 (13 lattice points) ...
Found 0 (non-independent) relations. Reducing ...
Calculating relations for level 1 (5 lattice points) ...
Found 0 (non-independent) relations.

Dimension estimate for empty skein part at level 1: 5.

Visualisation:

x x x 
- x x 
. | . 

Dimension estimate for empty skein part at level 2: 13.

Visualisation:

x x x x x 
. x x x x 
- - x x x 
. . | . x 
. . | . . 

Dimension estimate for empty skein part at level 3: 24.

Visualisation:

x x x x x x . 
. x x x x x x 
. . x x x x x 
- - - x x x x 
. . . |

In [6]:
data

(69 x 85 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field,
 69 x 85 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field,
 [5, 13, 24, 32, 31, 16])

In [11]:
print(data[1][1])

(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, (q^42 - q^39 + 3*q^24 - q^22 + q^21 - q^19 + q^13)/(q^49 - q^46 + 3*q^31 - q^29 + q^28 - q^27 - q^26 + q^20 - q^3 + q^2 - 2), (-q^35 - q^34 + q^32 - q^11 + q^9 - q^8 - 3*q^7 + 2*q^5)/(q^49 - q^46 + 3*q^31 - q^29 + q^28 - q^27 - q^26 + q^20 - q^3 + q^2 - 2), (q^45 + q^44 + q^43 + 3*q^27 + 2*q^26 + q^25 + 2*q^24 + 2*q^23 + q^22 + q^21 + q^20 + q^19 + q^18 + q^17 + 2*q^16 + 2*q^15 + 2*q^14 + 2*q^13 + 2*q^12 + 2*q^11 + 2*q^10 + 2*q^9 + 2*q^8 + 2*q^7 + 2*q^6 + 2*q^5 + 2*q^4 + 2*q^3 + q^2 + 2*q + 2)/(q^52 + q^51 + q^50 + 3*q^34 + 3*q^33 + 2*q^32 + 3*q^31 + 2*q^30 + q^29 + q^28 + q^27 + q^26 + q^25 + q^24 + 2*q^23 + 2*q^22 + 2*q^21 + 2*q^20 + 2*q^19 + 2*q^18 + 2*q^17 + 2*q^16 + 2*q^15 + 2*q^14 + 2*q^13 + 2*q^12 + 2*q^11 + 2*q^10 + 2*q^9 + 2*q^8 + 2*q^7 + q^6 + 2*q^5 + 2*q^4), (-q^34 + q^33 - q^31 - 2*q^30 + q^29 + 4*q^28 + q^27 - q^26 - 2*q^

In [16]:
print(data[0][:9][:41])

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             1                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [36]:
def compute_reduced_matrix_modded(gamma, shell_level, interactive_flag, base_level = 1, base_data=None):
    '''
    Computes the reduced matrix of relations at a given shell level, as well as
    its co-rank, which is an estimate of the dimension of the empty part of the
    skein module

    Returns a tuple (A, A_reduced, dim_estimates), where A is the matrix and
    dim_estimates is the list of estimated dimensions for shell levels <= to the
    shell level parameter.

    Works recursively: the relation matrix for level n is built up as

                [[  X  | 0 ]
                 [---------]
                 [relations]]

    where X is the matrix of relations for level n-1, in REF, obtained from a
    recursive call to the function, and the list relations is a list of
    relations at level n that were not already included at level n-1.

    Supports recursion to a specified base_level, with base_data the results of
    a previous call to the function (that is, a tuple of length 3). Usually this
    would be loaded as a sage object saved from a previous session. If the
    recursion base is unspecified then the default is recurse to shell_level 1
    and compute this data from scratch.
    '''

    # Base case.
    if shell_level == base_level:
        if base_level == 1: # Recursing all the way to level 1          
            # For each shell level, compute #{lattice points}.
            N = (2*shell_level + 1)*(shell_level + 1) - shell_level
            # And the number of points in previous level.
            M = (2*shell_level - 1)*(shell_level) - shell_level + 1
            if interactive_flag:
                print("Calculating relations for level %d (%d lattice points) ..." % (shell_level, N))

            # Get the relations new to this shell level.
            relations = get_new_relations_empty(gamma, shell_level, order_by_shell_level)
            if interactive_flag:
                print("Found %d (non-independent) relations.\n" % len(relations))
            # Form a relation matrix, and reduce it.
            A = matrix(K, relations, sparse=True, immutable=True)
            A_reduced = A.rref()

            # Get the spanning set
            ordering = order_by_shell_level(shell_level)
            spanning_set = get_spanning_set(A_reduced, ordering, shell_level)

            # Base of the recursion: return the reduced matrix and spanning set.
            dim_estimate = len(spanning_set)
            if interactive_flag:
                print("Dimension estimate for empty skein part at level %d: %d.\n\nVisualisation:\n" % (shell_level, dim_estimate))
                print_generators(shell_level, spanning_set, order_by_shell_level)
                
            print("HHIIII")

            return ([A], A_reduced, [dim_estimate])

        else: # Recursing to a higher level and picking up where we left off.
            if interactive_flag:
                print("Using previosuly calculated relations for level %d..." % (shell_level))
            return base_data

    else:
        print(shell_level)
        # For each shell level, compute #{lattice points}.
        N = (2*shell_level + 1)*(shell_level + 1) - shell_level
        # And the number of points in previous level.
        M = (2*shell_level - 1)*(shell_level) - shell_level + 1
        if interactive_flag:
            print("Calculating relations for level %d (%d lattice points) ..." % (shell_level, N))

        # Get the relations new to this shell level.
        relations = get_new_relations_empty(gamma, shell_level, order_by_shell_level)
        if interactive_flag:
            print("Found %d (non-independent) relations. Reducing ..." % len(relations))
        # Get the relation matrix for the previous shell level, recursively.
        prev_data = compute_reduced_matrix(gamma, shell_level - 1, interactive_flag, base_level, base_data)
        A_old = prev_data[1]
        dimensions = prev_data[2]

        # Handle the case when there were no relations in the previous level
        if len(A_old.rows()) == 0:
            A = matrix(K, relations, sparse=True, immutable=True)
        else:
            # Use the new relations and old, reduced matrix to build the relations
            # matrix for this shell level, and reduce.
            Zeros_right = zero_matrix(K, A_old.nrows(), N - M, sparse=True)
            A_upper = block_matrix([[A_old, Zeros_right]])
            A_lower = matrix(K, relations, sparse=True, immutable=True)
            A = block_matrix([[A_upper], [A_lower]])
            
        if shell_level != 5:
            A_reduced = A.rref()
        else:
            A_reduced = A

        # Get the spanning set
        ordering = order_by_shell_level(shell_level)
        spanning_set = get_spanning_set(A_reduced, ordering, shell_level)

        # Add the dimension estimate to the list, print output, and return
        dim_estimate = len(spanning_set)
        dimensions.append(dim_estimate)
        if interactive_flag:
            print("Dimension estimate for empty skein part at level %d: %d.\n\nVisualisation:\n" % (shell_level, dim_estimate))
            print_generators(shell_level, spanning_set, order_by_shell_level)

        print(type(prev_data[0]))
        return (prev_data[0].append(A), A_reduced, dimensions)

In [40]:
data = compute_reduced_matrix_mod(M, 5, True)

Calculating relations for level 5 (61 lattice points) ...
Found 21 (non-independent) relations. Reducing ...
Calculating relations for level 4 (41 lattice points) ...
Found 8 (non-independent) relations. Reducing ...
Calculating relations for level 3 (25 lattice points) ...
Found 1 (non-independent) relations. Reducing ...
Calculating relations for level 2 (13 lattice points) ...
Found 0 (non-independent) relations. Reducing ...
Calculating relations for level 1 (5 lattice points) ...
Found 0 (non-independent) relations.

Dimension estimate for empty skein part at level 1: 5.

Visualisation:

x x x 
- x x 
. | . 

Dimension estimate for empty skein part at level 2: 13.

Visualisation:

x x x x x 
. x x x x 
- - x x x 
. . | . x 
. . | . . 

Dimension estimate for empty skein part at level 3: 24.

Visualisation:

x x x x x x . 
. x x x x x x 
. . x x x x x 
- - - x x x x 
. . . | . x x 
. . . | . . x 
. . . | . . . 

Dimension estimate for empty skein part at level 4: 32.

Visualisation

In [42]:
A = data[0]

In [44]:
A

30 x 61 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field (use the '.str()' method to see the entries)

In [43]:
import time

In [49]:
methods = ['classical','partial_pivoting','scaled_partial_pivoting','scaled_partial_pivoting_valuation','strassen','default']

In [52]:
A

30 x 61 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field (use the '.str()' method to see the entries)

In [54]:
A.rref()

30 x 61 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field (use the '.str()' method to see the entries)

In [79]:
A = matrix(K, get_relations_empty(M, 6, order_by_shell_level).values())

In [80]:
A

69 x 85 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field (use the '.str()' method to see the entries)

In [81]:
times = []
for m in methods:
    t_0 = time.time()
    A.echelon_form(algorithm=m)
    t_1 = time.time()
    times.append(t_1 - t_0)
    print(m)

classical
partial_pivoting
scaled_partial_pivoting
scaled_partial_pivoting_valuation
strassen
default


In [82]:
times

[2994.6566734313965,
 7.3909759521484375e-06,
 1.430511474609375e-06,
 9.5367431640625e-07,
 1.9073486328125e-06,
 1.9073486328125e-06]

In [83]:
### BIG PROBLEM: I think we are not saving the non RREF form of the matrix in compute_reduced_matrix?? 